<a href="https://colab.research.google.com/github/Scott-S-Lin/NTUT_PhD/blob/main/Lab8_scott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 實驗：實作InceptionV3網路架構

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/taipeitechmmslab/MMSLAB-TF2/blob/master/Lab8.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/taipeitechmmslab/MMSLAB-TF2/blob/master/Lab8.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

### Import必要套件

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

---
## Keras Applications
### 創建InceptionV3網路架構
- 輸入大小(預設)：(299, 299, 3)
- 權重(預設)：`imagenet`
- 輸出類別(預設)：1000個類別

In [ ]:
model = tf.keras.applications.InceptionV3(include_top=True, weights='imagenet')

透過`model.summary`可以察看網路模型的每一層資訊：

In [ ]:
model.summary()

將網路模型儲存到TensorBoard上：

In [ ]:
model_tb = tf.keras.callbacks.TensorBoard(log_dir='lab8-logs-inceptionv3-keras')
model_tb.set_model(model)

In [ ]:
%reload_ext tensorboard

In [ ]:
tensorboard --logdir lab8-logs-inceptionv3-keras/

### 資料前處理和輸出解碼

使用別人提供的模型預測，需要注意兩件事情，1)訓練時的資料前處理，2)輸出結果對應到的類別。

Keras很貼心的提供每個模型相對應的資料預處理和輸出解碼的函式。
- preprocess_input：網路架構的影像前處理(注意：每一個模型在訓練時做的資料正規化並不會相同，例如：VGG、ResNet-50輸入影像為0~255的數值，而inception_v3、xception輸入影像為-1~1的數值)。
- decode_predictions：對應網路架構的輸出解碼。

Import資料預處理和輸出解碼的函式：

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.inception_v3 import decode_predictions

### 預測輸出結果

創建影像讀取的函式：讀取影像，並將影像大小縮放大299x299x3的尺寸。

In [ ]:
def read_img(img_path, resize=(299,299)):
    img_string = tf.io.read_file(img_path)  # 讀取檔案
    img_decode = tf.image.decode_image(img_string)  # 將檔案以影像格式來解碼
    img_decode = tf.image.resize(img_decode, resize)  # 將影像resize到網路輸入大小
    # 將影像格式增加到4維(batch, height, width, channels)，模型預測要求格式
    img_decode = tf.expand_dims(img_decode, axis=0)
    return img_decode

In [ ]:
%pwd

從資料夾中讀取一張影像（elephant.jpg）作為測試：

In [ ]:
img_path = 'image/elephant.jpg'
img = read_img(img_path)  # 透過剛創建的函式讀取影像
plt.imshow(tf.cast(img, tf.uint8)[0])  # 透過matplotlib顯示圖片需將影像轉為Integers

預測結果：

In [ ]:
img = preprocess_input(img)  # 影像前處理
preds = model.predict(img)  # 預測圖片
print("Predicted:", decode_predictions(preds, top=3)[0])  # 輸出預測最高的三個類別

---
## TensorFlow Hub

Install:
```
pip install tensorflow-hub
```

Search:
https://tfhub.dev/

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

### 創建Inception V3模型

Model:
https://tfhub.dev/google/tf2-preview/inception_v3/classification/2

num_classes = 1001 classes of the classification from the original training
 
Image:height x width = 299 x 299 pixels, 3 RGB color values in the range 0~1

labels file: https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt

In [ ]:
# Inception V3預訓練模型的URL
module_url = "https://tfhub.dev/google/tf2-preview/inception_v3/classification/4"

# 創建一個Sequential Model，網路模型裡面包含了Inception V3網路層
model = tf.keras.Sequential([
    # hub.KerasLayer將載入的Inception V3模型封裝成網路層(Keras Layer)
    hub.KerasLayer(module_url, 
                  input_shape=(299, 299, 3),  # 模型輸入大小
                   output_shape=(1001, ),   # 模型輸出大小
                   name='Inception_v3')  # 網路層名稱
])

In [ ]:
model.summary()

### 資料前處理和輸出解碼

創建資料前處理函式：

In [ ]:
def read_img(img_path, resize=(299,299)):
    img_string = tf.io.read_file(img_path)  # 讀取檔案
    img_decode = tf.image.decode_image(img_string)  # 將檔案以影像格式來解碼
    img_decode = tf.image.resize(img_decode, resize)  # 將影像resize到網路輸入大小
    img_decode = img_decode / 255.0  # 對影像做正規畫，將數值縮放到0~1之間
    # 將影像格式增加到4維(batch, height, width, channels)，模型預測要求格式
    img_decode = tf.expand_dims(img_decode, axis=0)  # 
    return img_decode

創建輸出解碼器：

In [ ]:
# 下載ImageNet 的標籤檔
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
# 讀取標籤檔中的數據
with open(labels_path) as file:
    lines = file.read().splitlines()
print(lines)  # 顯示讀取的標籤

imagenet_labels = np.array(lines)  # 將標籤轉成numpy array做為網路輸出的解碼器

### 預測輸出結果

從資料夾中讀取一張影像（elephant.jpg）作為測試：

In [ ]:
img_path = 'image/elephant.jpg'
img = read_img(img_path)  # 透過剛創建的函式讀取影像
plt.imshow(img[0])

預測結果：

In [ ]:
preds = model.predict(img)  # 預測圖片
index = np.argmax(preds)  # 取得預測結果最大的Index
print("Predicted:", imagenet_labels[index])  # 透過解碼器將輸出轉成標籤

顯示最好的三個預測：

In [ ]:
# 取得預測結果最大的三個indexs
top3_indexs = np.argsort(preds)[0, ::-1][:3]  
print("Predicted:", imagenet_labels[top3_indexs])  # 透過解碼器將輸出轉成標籤